In [ ]:
!pip install pdfservices-sdk==2.3.1
!pip install docling
!pip install mistralai
!pip install openpyxl 

In [27]:
import json
import os
import os.path
import re
import sys
import zipfile
from datetime import datetime
from pathlib import Path

import openpyxl
import pandas as pd
import requests
from dotenv import load_dotenv
from mistralai import DocumentURLChunk, Mistral
from mistralai.models import OCRResponse

load_dotenv()

sys.path.append("..")
from function.adobe import adobeLoader, extract_text_from_file_adobe, get_dict_xlsx

ADOBE PDF Extractions API

In [ ]:
client_id = os.getenv("ADOBE_CLIENT_ID")
client_secret = os.getenv("ADOBE_CLIENT_SECRET")
# Adobe output zip file path
input_pdf = "../data/raw/Interim-Report-2024.pdf"
output_zip_path = "../data/processed/adobe_result/Interim-Report-2024/sdk.zip"
output_zipextract_folder = "../data/processed/adobe_result/Interim-Report-2024/"
# Run adobe API
adobeLoader(
    input_pdf, output_zip_path, client_id=client_id, client_secret=client_secret
)
# unzip adobe output zipfile, extract text and table from adobe output zip file
df, json_strings = extract_text_from_file_adobe(
    output_zip_path, output_zipextract_folder
)

In [ ]:
output_folder = "../data/processed/adobe_result/Interim-Report-2024/tables"
xlsx_files = [f for f in os.listdir(output_folder) if f.endswith(".xlsx")]
for xlsx_file in xlsx_files:
    print(f"\nTable from {xlsx_file}:")
    data_dict = get_dict_xlsx(output_folder, xlsx_file)
    df = pd.DataFrame(data_dict)
    print(df.to_markdown(index=False))
    print("\n---")


Table from fileoutpart10.xlsx:
| Amounts in millions of euros                             | June 30, 2024    | December 31, 2023    |
|:---------------------------------------------------------|:-----------------|:---------------------|
| Gross carrying amount loans and advances to customers    | 467,704          | 453,550              |
| Hedge accounting adjustment                              | (11,175)         | (11,379)             |
| Impairment allowances on loans and advances to customers | (2,897)          | (2,909)              |
| Total loans and advances to customers                    | 453,632          | 439,262              |

---

Table from fileoutpart6.xlsx:
| Amounts in millions of euros                                   |   First half-year 2024  | First half-year 2023    |
|:---------------------------------------------------------------|------------------------:|:------------------------|
| Additions and releases of provisions                           |          

In [ ]:
# create output md file
md_output_path = "../data/processed/adobe_result/Interim-Report-2024/markdown_tables.md"
with open(md_output_path, "w") as f:
    for xlsx_file in xlsx_files:
        f.write(f"\n## Table from {xlsx_file}:\n\n")
        data_dict = get_dict_xlsx(output_folder, xlsx_file)
        df = pd.DataFrame(data_dict)
        f.write(df.to_markdown(index=False))


Agentic document extraction Landing AI

In [9]:
api_key = os.getenv("LANDINGAI_API_KEY")
input_pdf = "../data/raw/Interim-Report-2024.pdf"

url = "https://api.va.landing.ai/v1/tools/agentic-document-analysis"
files = {"pdf": open(f"{input_pdf}", "rb")}
headers = {
    "Authorization": f"Basic {api_key}",
}
response = requests.post(url, files=files, headers=headers)

print(response.json())

{'data': {'markdown': '## Balance Sheet Developments <!-- text, from page 0 (l=0.037,t=0.113,r=0.198,b=0.135), with ID 40e3f665-89bb-438f-a64e-86a9d22d7fa1 -->\n\n## Balance Sheet\n\n### Amounts in billions of euros\n\n<table>\n  <tr>\n    <th></th>\n    <th>06-30-2024</th>\n    <th>12-31-2023</th>\n  </tr>\n  <tr>\n    <td>Cash and cash equivalents</td>\n    <td>80.6</td>\n    <td>90.5</td>\n  </tr>\n  <tr>\n    <td>Loans and advances to customers</td>\n    <td>453.6</td>\n    <td>439.3</td>\n  </tr>\n  <tr>\n    <td>Financial assets</td>\n    <td>24.2</td>\n    <td>20.4</td>\n  </tr>\n  <tr>\n    <td>Loans and advances to banks</td>\n    <td>25.9</td>\n    <td>26.5</td>\n  </tr>\n  <tr>\n    <td>Derivatives</td>\n    <td>21.0</td>\n    <td>22.0</td>\n  </tr>\n  <tr>\n    <td>Other assets</td>\n    <td>16.3</td>\n    <td>15.1</td>\n  </tr>\n  <tr>\n    <td><strong>Total assets</strong></td>\n    <td><strong>621.6</strong></td>\n    <td><strong>613.8</strong></td>\n  </tr>\n  <tr>\n   

In [25]:
api_key = os.getenv("LANDINGAI_API_KEY")
input_pdf = "../data/raw/Our-Impact-in-2023.pdf"

url = "https://api.va.landing.ai/v1/tools/agentic-document-analysis"
files = {"pdf": open(f"{input_pdf}", "rb")}
headers = {
    "Authorization": f"Basic {api_key}",
}
response = requests.post(url, files=files, headers=headers)

print(response.json())

{'data': {'markdown': '## Page Header\n\n- **Introduction**\n- **About This Report**\n- **Foreword**\n- **Rabobank at a Glance**\n- **About Us**\n- **Value Creation**\n- **Better World** (highlighted in orange)\n- **Better Bank**\n- **Appendix**\n- **Disclaimer** <!-- page_header, from page 0 (l=0.037,t=0.025,r=0.728,b=0.039), with ID 817e48b1-616f-423a-b8b8-7a0e1f39f0b2 -->\n\n## Financed Emissions and Avoided Emissions Overview\n\n### Financed Emissions in Mt CO2e\n\n- **2022 Financed Emissions**: 48.8 Mt CO2e\n- **2021 Financed Emissions**: 51.5 Mt CO2e\n\n### Avoided Emissions via Renewable Energy Portfolio\n\n- **2022 Avoided Emissions**: 5.3 Mt CO2e\n- **2021 Avoided Emissions**: 4.8 Mt CO2e\n\n### Emissions Breakdown by Sector\n\n#### Loans to Private Individuals\n\n- **Residential Real Estate**:\n  - 2022: 1.9 Mt CO2e\n  - 2021: 2.0 Mt CO2e\n\n#### Dutch Business Clients\n\n- **Food & Agriculture**:\n  - 2022: 9.3 Mt CO2e\n  - 2021: 10.0 Mt CO2e\n- **Trade, Industry & Services*

In [26]:
# extract the md content from the response
markdown_content = response.json()["data"]["markdown"]
output_md = "../data/processed/landingai_result/Our-Impact-in-2023.md"
os.makedirs(os.path.dirname(output_md), exist_ok=True)

# save the md output
with open(output_md, "w") as f:
    f.write(markdown_content)

Docling

In [12]:
from docling.document_converter import DocumentConverter

# input and output paths
input_pdf = "../data/raw/Interim-Report-2024.pdf"
output_md = "../data/processed/docling_result/Interim-Report-2024.md"

# Create output directory if it doesn't exist
os.makedirs(os.path.dirname(output_md), exist_ok=True)

# convert the PDF to md
print(f"Converting {input_pdf} with Docling")
converter = DocumentConverter()
result = converter.convert(input_pdf)

# save  md output
with open(output_md, "w") as f:
    f.write(result.document.export_to_markdown())

print(f"Docling output saved to {output_md}")

Converting ../data/raw/Interim-Report-2024.pdf with Docling


Docling output saved to ../data/processed/docling_result/Interim-Report-2024.md


In [28]:
from docling.document_converter import DocumentConverter

# input and output paths
input_pdf = "../data/raw/Our-Impact-in-2023.pdf"
output_md = "../data/processed/docling_result/Our-Impact-in-2023.md"

# create output directory if it doesn't exist
os.makedirs(os.path.dirname(output_md), exist_ok=True)

# convert the PDF to md
print(f"Converting {input_pdf} with Docling")
converter = DocumentConverter()
result = converter.convert(input_pdf)

# save md output
with open(output_md, "w") as f:
    f.write(result.document.export_to_markdown())

print(f"Docling output saved to {output_md}")

Converting ../data/raw/Our-Impact-in-2023.pdf with Docling
Docling output saved to ../data/processed/docling_result/Our-Impact-in-2023.md


MISTRAL OCR

In [ ]:
api_key = os.getenv("MISTRAL_API_KEY")
client = Mistral(api_key=api_key)


def replace_images_in_markdown(markdown_str: str, images_dict: dict) -> str:
    for img_name, base64_str in images_dict.items():
        markdown_str = markdown_str.replace(
            f"![{img_name}]({img_name})", f"![{img_name}]({base64_str})"
        )
    return markdown_str


def get_combined_markdown(ocr_response: OCRResponse) -> str:
    markdowns: list[str] = []
    for page in ocr_response.pages:
        image_data = {}
        for img in page.images:
            image_data[img.id] = img.image_base64
        markdowns.append(replace_images_in_markdown(page.markdown, image_data))

    return "\n\n".join(markdowns)


# input and output paths
input_pdf = "../data/raw/Interim-Report-2024.pdf"
output_md = "../data/processed/mistral_result/Interim-Report-2024.md"

# create output directory if it doesn't exist
os.makedirs(os.path.dirname(output_md), exist_ok=True)
with open(input_pdf, "rb") as f:
    uploaded_file = client.files.upload(
        file={
            "file_name": Path(input_pdf).stem,
            "content": f.read(),
        },
        purpose="ocr",
    )

# get signed URL
signed_url = client.files.get_signed_url(file_id=uploaded_file.id, expiry=1)

# process with OCR
pdf_response = client.ocr.process(
    document=DocumentURLChunk(document_url=signed_url.url),
    model="mistral-ocr-latest",
    include_image_base64=True,
)

# generate markdown
markdown = get_combined_markdown(pdf_response)

# save the markdown output
with open(output_md, "w") as f:
    f.write(markdown)

print(f"Mistral output saved to {output_md}")

Mistral output saved to ../data/processed/mistral_result/Interim-Report-2024.md


In [24]:
import os
from pathlib import Path

from dotenv import load_dotenv
from mistralai import DocumentURLChunk, Mistral
from mistralai.models import OCRResponse

load_dotenv()

api_key = os.getenv("MISTRAL_API_KEY")
client = Mistral(api_key=api_key)


def replace_images_in_markdown(markdown_str: str, images_dict: dict) -> str:
    for img_name, base64_str in images_dict.items():
        markdown_str = markdown_str.replace(
            f"![{img_name}]({img_name})", f"![{img_name}]({base64_str})"
        )
    return markdown_str


def get_combined_markdown(ocr_response: OCRResponse) -> str:
    markdowns: list[str] = []
    for page in ocr_response.pages:
        image_data = {}
        for img in page.images:
            image_data[img.id] = img.image_base64
        markdowns.append(replace_images_in_markdown(page.markdown, image_data))

    return "\n\n".join(markdowns)


# Set the input and output paths
input_pdf = "../data/raw/Our-Impact-in-2023.pdf"
output_md = "../data/processed/mistral_result/Our-Impact-in-2023.pdf.md"

# Create output directory if it doesn't exist
os.makedirs(os.path.dirname(output_md), exist_ok=True)

# Upload file to Mistral
with open(input_pdf, "rb") as f:
    uploaded_file = client.files.upload(
        file={
            "file_name": Path(input_pdf).stem,
            "content": f.read(),
        },
        purpose="ocr",
    )

# Get signed URL
signed_url = client.files.get_signed_url(file_id=uploaded_file.id, expiry=1)

# Process with OCR
pdf_response = client.ocr.process(
    document=DocumentURLChunk(document_url=signed_url.url),
    model="mistral-ocr-latest",
    include_image_base64=True,
)

# Generate markdown
markdown = get_combined_markdown(pdf_response)

# Save the markdown output
with open(output_md, "w") as f:
    f.write(markdown)

print(f"Mistral output saved to {output_md}")

Mistral output saved to ../data/processed/mistral_result/Our-Impact-in-2023.pdf.md
